In [3]:
#!/public/local/anaconda2/bin/python
# -*- coding: utf-8 -*-

import numpy as np
import os
import glob
import sys

# settings
# big endian
is_sys_little = sys.byteorder == 'little'
is_watershed_mask = True
data_type = '>f4'
nx = 450
ny = 270
len_fill_dir = 3
# cut sizes
# nbr_cell_num = 1
nbr_cell_num = 1

# dirs
proj_path = os.path.abspath('.')
os.chdir(proj_path)
in_dir = 'sbcs'
# out_dir = 'wshed_split'
out_dir = 'sbcs_32'
in_filename_list = glob.glob(in_dir + os.sep + '*')
# WARNING: for linux, MUST sort before used
in_filename_list.sort()

# mask: 1: watershed; -999: nodata
if is_watershed_mask:
    wshed_mask = 'mask'
    mask_arr = np.fromfile(os.path.join(proj_path, wshed_mask), dtype=data_type).reshape(ny, nx)
    mask_inside = np.where(mask_arr == 1, True, False)
    print ('number of grids in watershed: ', mask_inside.sum())
else:
    mask_inside = np.ones((ny, nx), dtype=np.bool)
    print ('number of grids in whole region: ', mask_inside.sum())

for in_filename in in_filename_list:
#    print in_filename

    # read
    arr = np.fromfile(os.path.join(proj_path, in_filename), dtype=data_type).reshape(ny, nx)

    for ix_start in range(0, nx, nbr_cell_num):
        ix_end = min(ix_start + nbr_cell_num, nx)
        for iy_start in range(0, ny, nbr_cell_num):
            iy_end = min(iy_start + nbr_cell_num, ny)

            # masked?
            if not np.any(mask_inside[iy_start:iy_end, ix_start:ix_end]):
                continue

            # create dirs
            split_dir = str(ix_start + 1).zfill(len_fill_dir) + 'x' + str(iy_start + 1).zfill(len_fill_dir) +  \
                '_' + str(nbr_cell_num)
            if not os.path.isdir(os.path.join(out_dir, split_dir)):
                os.mkdir(os.path.join(out_dir, split_dir))
            out_filepath = os.path.join(out_dir, in_filename.replace(in_dir, split_dir))

            # cut
            arr_cut = arr[iy_start:iy_end, ix_start:ix_end]
            # byte order swap
            # here data_type contains big endian '>', so no need to change.

            # save
            arr_cut.tofile(out_filepath)

print ('Done!')


number of grids in watershed:  27
Done!
